# SVR model

## Import libraries and download the data

In [6]:
#Download libraries
import re
#from collections import defaultdict
import pandas as pd
#import altair as alt
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA
#from sklearn.svm import SVR
#from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import OneHotEncoder
#from preprocessing import preprocessing_na, clean_categorical
#import time
from drop import drop_columns
from PCA_data import *

In [7]:
#Download the data
df = pd.read_csv('../data/train_data.zip')
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1807558,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,37.0,418.67,0.116368,0.025516,0.090852
1,1806729,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,63.0,710.00,0.175774,0.020170,0.155604
2,MR00096403,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,15.0,241.24,0.112038,0.006893,0.105145
3,1806350,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,45.0,468.51,0.120088,0.003619,0.116469
4,FM00169477,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,83.0,326.05,0.132935,0.025298,0.107637


In [15]:
categorical_features = df.loc[:, df.dtypes == "object"].columns
df.loc[:,categorical_features]

,external_id,monthly_weekday_counts,monthly_survey,name,date,city,state,country,closest_place_category,income_class,density_class,climate,county
0,1807558,NaN,NaN,NaN,NaN,Columbia,South Carolina,USA,amenity,LI,LD,C,Richland
1,1806729,NaN,NaN,NaN,NaN,Covington,Washington,USA,office,HI,MD,C,King
2,MR00096403,NaN,NaN,NaN,NaN,Knoxville,Iowa,USA,place_of_worship,MI,LD,D,Marion
3,1806350,NaN,NaN,NaN,NaN,Madison,Wisconsin,USA,NaN,MI,MD,D,Dane
4,FM00169477,NaN,NaN,NaN,NaN,Savannah,Georgia,USA,NaN,LI,MD,C,Chatham
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50115,MR00111485,NaN,NaN,NaN,NaN,Dallas,Texas,USA,place_of_worship,MI,HD,C,Dallas
50116,Kyle Rec Kenner 4,NaN,NaN,NaN,NaN,Kenner,Louisiana,USA,NaN,LI,HD,C,Jefferson
50117,1901693,NaN,NaN,NaN,NaN,Sammamish,Washington,USA,NaN,HI,MD,C,King
50118,FM00166877,NaN,NaN,NaN,NaN,Livonia,Michigan,USA,NaN,HI,MD,D,Wayne


In [5]:
clean_data = get_pca_trans_data(df, 0.95)

ValueError: could not convert string to float: 'MR00096403'

# SVR Model

## Without categorical values

## With raw data

I am waiting for the preprocessing to be done to run this part

In [30]:
X_train, X_valid, y_train, y_valid = train_test_split(clean_data.drop(columns=['unacast_session_count', 'state', 'external_id']),
                                                    clean_data[['unacast_session_count']],
                                                    test_size=0.2,
                                                    random_state=2020)

In [7]:
#Fit basic SVR with default values
t0 = time.time()
svr = SVR()
svr.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()

In [8]:
#Find the train and validation score
print(svr.score(X_train, y_train.to_numpy().ravel())
svr.score(X_valid, y_valid.to_numpy().ravel())

-0.011561882561027748

In [10]:
(t1 - t0)/60

36.732024590174355

In [12]:
print(svr.score(X_train, y_train.to_numpy().ravel()))
#Took roughly 1h30

-0.023977981444002427


SVR takes a long time to run, and gave really poor results. We will keep it aside for now, and see latter if we want to use it or not.

I kept the following code for when we will have more computational power to run SVR.

In [ ]:
"""
#Kernel = ‘linear’
t0 = time.time()
parameters = {'C':np.logspace(-5,5,40), 'gamma':np.logspace(-10,10,40)}
svr = SVR(kernel='linear')
svr_opt = RandomizedSearchCV(svr, parameters, cv=5, iid=False, n_iter=100)
svr_opt.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()
print('The best parameter combination is {}.'.format(svc_opt.best_params_))
print('The accuracy on the test split is {:.2f}.'.format(svc_opt.score(X_test,y_test)))
total = t1-t0
print(total)
"""

## Catboost

### Raw data

In [46]:
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

In [32]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False)
# Fit model
t0 = time.time()
model.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()

In [36]:
(t1-t0)/60

4.0600755453109745

In [34]:
# Get R2 score
model.score(X_train, y_train.to_numpy().ravel())

0.9926611881964983

In [35]:
model.score(X_valid, y_valid.to_numpy().ravel())

0.45364117152820127

### Normalized data

In [44]:
#Drop categorical features
categorical_features = X_train.loc[:, clean_data.dtypes == "object"]
categorical_features.columns

Index([], dtype='object')

In [48]:
#Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.drop(columns = list(categorical_features.columns)))
X_valid_scaled = scaler.transform(X_valid.drop(columns = list(categorical_features.columns)))

In [50]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False)
# Fit model
t0 = time.time()
model.fit(X_train_scaled, y_train.to_numpy().ravel())
t1 = time.time()

In [51]:
(t1-t0)/60

2.829195515314738

In [52]:
# Get R2 score
model.score(X_train_scaled, y_train.to_numpy().ravel())

0.9926611881964983

In [53]:
model.score(X_valid_scaled, y_valid.to_numpy().ravel())

0.45364117152820127

### Log transformed response

In [37]:
y_train_log = np.log(np.where(y_train==0, 1, y_train))
y_valid_log = np.log(np.where(y_valid==0, 1, y_valid))

In [39]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False)
# Fit model
t0 = time.time()
model.fit(X_train, y_train_log.ravel())
t1 = time.time()

In [40]:
(t1-t0)/60

219.60970449447632

In [42]:
# Get R2 score
model.score(X_train, y_train_log.ravel())

0.9331527098740046

In [43]:
model.score(X_valid, y_valid_log.ravel())

0.7715039322762

## Hyperparameters tuning : Random search (Waiting for the EC2 to run the rest)

### Normalized data

In [64]:
catboost = CatBoostRegressor(verbose=False, loss_function='MAE')
parameters = {'iterations':range(1,1000,100), 'learning_rate':np.logspace(-10,0,40), 'depth':range(1,100,10)}
catboost_opt = RandomizedSearchCV(catboost, parameters, cv=5, iid=False, n_iter=5)
catboost_opt.fit(X_train_scaled, y_train.to_numpy().ravel())

KeyboardInterrupt: 

In [ ]:
print('The best parameter combination is {}.'.format(catboost_opt.best_params_))
print('The accuracy on the validation split is {:.2f}.'.format(catboost_opt.score(X_valid_scaled,y_valid)))

### Log transformation of the response

In [ ]:
catboost = CatBoostRegressor(verbose=False, loss_function='MAE')
parameters = {'iterations':range(1,1000,100), 'learning_rate':np.logspace(-10,0,40), 'depth':range(1,100,10)}
catboost_opt = RandomizedSearchCV(catboost, parameters, cv=5, iid=False, n_iter=5)
catboost_opt.fit(X_train, y_train_log)

In [ ]:
print('The best parameter combination is {}.'.format(catboost_opt.best_params_))
print('The accuracy on the validation split is {:.2f}.'.format(catboost_opt.score(X_valid,y_valid_log)))

### Log transformation of the response + Normalization

In [ ]:
catboost = CatBoostRegressor(verbose=False, loss_function='MAE')
parameters = {'iterations':range(1,1000,100), 'learning_rate':np.logspace(-10,0,40), 'depth':range(1,100,10)}
catboost_opt = RandomizedSearchCV(catboost, parameters, cv=5, iid=False, n_iter=5)
catboost_opt.fit(X_train_scaled, y_train_log)

In [ ]:
print('The best parameter combination is {}.'.format(catboost_opt.best_params_))
print('The accuracy on the validation split is {:.2f}.'.format(catboost_opt.score(X_valid_scaled,y_valid_log)))